In [ ]:
import os
import pandas as pd
import pyarrow.parquet as pq
import shutil

def transferir_dados_pacientes_para_curated(diretorio_raw, diretorio_curated, chunksize=1000):
    # Cria o diretório de curated caso não exista
    os.makedirs(diretorio_curated, exist_ok=True)

    # Lista de arquivos Parquet no diretório raw
    arquivos_raw = [f for f in os.listdir(diretorio_raw) if f.endswith('.parquet')]

    if not arquivos_raw:
        print("🚨 Nenhum arquivo Parquet encontrado no diretório raw.")
        return

    # Itera sobre os arquivos Parquet no diretório raw
    for arquivo in arquivos_raw:
        caminho_arquivo_raw = os.path.join(diretorio_raw, arquivo)
        
        # Lê o arquivo Parquet com pyarrow
        parquet_file = pq.ParquetFile(caminho_arquivo_raw)

        # Define o caminho de saída no diretório curated
        caminho_saida = os.path.join(diretorio_curated, arquivo)

        # Variável para controlar se o arquivo já foi criado
        arquivo_criado = False

        # Itera sobre os pedaços (chunks) do arquivo Parquet
        for i in range(0, parquet_file.num_row_groups, chunksize):
            # Leitura do lote de dados
            df_chunk = parquet_file.read_row_group(i).to_pandas()

            # ✅ Filtra os registros de pacientes no chunk
            df_pacientes = df_chunk[df_chunk['resource_resourceType'] == 'Patient']

            if not df_pacientes.empty:
                # Se o arquivo ainda não foi criado, cria o arquivo Parquet inicial
                if not arquivo_criado:
                    df_pacientes.to_parquet(caminho_saida, engine='pyarrow', index=False)
                    print(f"✅ Arquivo de pacientes salvo em: {caminho_saida}")
                    arquivo_criado = True
                else:
                    # Caso o arquivo já tenha sido criado, append dos dados
                    df_pacientes.to_parquet(caminho_saida, engine='pyarrow', index=False, compression='snappy', append=True)
                    print(f"✅ Lote de pacientes adicionado em: {caminho_saida}")

        # Opcional: Mover o arquivo original para outro diretório (se necessário)
        # shutil.move(caminho_arquivo_raw, 'diretorio_backup')  # Caso queira mover o arquivo original para backup

# 🔥 Exemplo de uso
diretorio_raw = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\raw'    # Substitua pelo caminho real
diretorio_curated = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient'    # Substitua pelo caminho real
chunksize = 1000  # Tamanho do chunk (lote)

# Chama a função para transferir os dados
transferir_dados_pacientes_para_curated(diretorio_raw, diretorio_curated, chunksize)

✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_001.parquet
✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_002.parquet
✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_003.parquet
✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_004.parquet
✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_005.parquet
✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_006.parquet
✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\patient\lote_007.parquet
✅ Arquivo de pacientes salvo em: C:\Users\Leonardo\OneD